Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [1]:
# Data cleaning
import pandas as pd
import os

full_df = pd.read_csv(os.getcwd()+"/data/bike-no.csv")
full_df['new_date'] = pd.to_datetime(full_df['date'], format = "%m/%d/%Y")

# Filter by date
train_df = full_df.loc[full_df['new_date'] < '2012-09-1'].copy()
test_df = full_df.loc[(full_df['new_date'] >= '2012-09-1')].copy()
print(train_df.shape," ",test_df.shape)
print(train_df['new_date'].max()," ",test_df['new_date'].min())

train_df['date'] = train_df['new_date'].dt.strftime('%Y-%m-%d')
test_df['date'] = test_df['new_date'].dt.strftime('%Y-%m-%d')

train_df.drop(columns=['new_date'],inplace = True)
test_df.drop(columns=['new_date'],inplace = True)

(609, 15)   (122, 15)
2012-08-31 00:00:00   2012-09-01 00:00:00


In [2]:
train_df.to_csv(os.getcwd()+"/data/training-mltable-folder/bike-no-train.csv",index = False)
test_df.to_csv(os.getcwd()+"/data/test-mltable-folder/bike-no-test.csv",index = False)

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/auto-ml-forecasting-bike-share.png)

# Automated Machine Learning
**BikeShare Demand Forecasting**

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Compute](#Compute)
1. [Data](#Data)
1. [Train](#Train)
1. [Featurization](#Featurization)
1. [Evaluate](#Evaluate)

## Introduction
This notebook demonstrates demand forecasting for a bike-sharing service using AutoML.

AutoML highlights here include built-in holiday featurization, accessing engineered feature names, and working with the `forecast` function. Please also look at the additional forecasting notebooks, which document lagging, rolling windows, forecast quantiles, other ways to use the forecast function, and forecaster deployment.

Make sure you have executed the [configuration notebook](../../../configuration.ipynb) before running this notebook.

Notebook synopsis:
1. Creating an Experiment in an existing Workspace
2. Configuration and local run of AutoML for a time-series model with lag and holiday features 
3. Viewing the engineered names for featurized data and featurization summary for all raw features
4. Evaluating the fitted model using a rolling test 

## Setup

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries


In [3]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl
from azure.ai.ml import Input

import json
import logging
from datetime import datetime
import numpy as np
import pandas as pd

This notebook is compatible with Azure ML SDK version 1.35.0 or later.

As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [4]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91" #"<SUBSCRIPTION_ID>"
    resource_group =  "sagoswami_southcentralus_rg" #<RESOURCE_GROUP>"
    workspace =  'test'#'amlb69723617'#"sagoswami_ws" #<AML_WORKSPACE_NAME>"
    
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


#### Show Azure ML Workspace information

In [5]:
import pandas as pd

workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

,
Workspace,test
Subscription ID,381b38e9-9840-4719-a5a0-61d9585e1e91
Resource Group,sagoswami_southcentralus_rg
Location,eastus2


## Compute
You will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

## 2. Data

The [Machine Learning service workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-workspace) is paired with the storage account, which contains the default data store. We will use it to upload the bike share data and create [tabular dataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py) for training. A tabular dataset defines a series of lazily-evaluated, immutable operations to load data from the data source into tabular representation.

With Azure Machine Learning MLTables you can keep a single copy of data in your storage, easily access data during model training, share data and collaborate with other users. 
Below, we will upload the data by creating an MLTable to be used for training.

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

In [6]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

# WITH REMOTE PATH
# my_training_data_input  = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my-forecasting-mltable")

In [7]:
import os
import shutil

os.makedirs("test_dataset", exist_ok=True)
shutil.copy(
    "data/test-mltable-folder/bike-no-test.csv",
    "test_dataset/bike-no-test.csv",
)

my_test_data_input = Input(
    type=AssetTypes.URI_FOLDER,
    path="test_dataset/",
)

We will upload the dirrectory with the test set so that further we will be able to run the inferencing on the batch endpoint.

In [8]:
my_training_data_input

{'type': 'mltable', 'path': './data/training-mltable-folder'}

## 3. Create or Attach existing AmlCompute.
[Azure Machine Learning Compute](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) is a managed-compute infrastructure that allows the user to easily create a single or multi-node compute. In this tutorial, you create AmlCompute as your training compute resource.

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [9]:
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.ml.entities import AmlCompute

cluster_name = "bike-share-v2"

try:
    # Retrieve an already attached Azure Machine Learning Compute.
    compute = ml_client.compute.get(cluster_name)
except ResourceNotFoundError as e:
    compute = AmlCompute(
        name=cluster_name,
        size="STANDARD_DS12_V2",
        type="amlcompute",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=120,
    )
    poller = ml_client.begin_create_or_update(compute)
    poller.wait()

## 4. Configure and run the AutoML Forecasting training job
In this section we will configure and run the AutoML job, for training the model.

### 4.1 Configure the job through the forecasting() factory function

#### forecasting() function parameters:

The `forecasting()` factory function allows user to configure AutoML for the forecasting task for the most common scenarios with the following properties.

|Property|Description|
|-|-|
|**target_column_name**|The name of the label column.|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**training_data**|The training data to be used within the experiment. You can use a registered MLTable in the workspace using the format '&lt;mltable_name&gt;:&lt;version\&gt;' OR you can use a local file or folder as a MLTable. For e.g Input(mltable='my_mltable:1') OR Input(mltable=MLTable(local_path="./data")) The parameter 'training_data' must always be provided.|
|**compute**|The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it any other compute in the workspace.|
|**n_cross_validations**|Number of cross-validation folds to use for model/pipeline selection. The default value is "auto", in which case AutoMl determines the number of cross-validations automatically, if a validation set is not provided. Or users could specify an integer value.|
|**name**|The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
|**experiment_name**|The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.|
|**enable_model_explainability**|If set to true, the explanations for the best model will be generated.|

### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.

|Property|Description|
|-|-|
|**timeout_minutes**|Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.|
|**trial_timeout_minutes**|Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.|
|**max_trials**|The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.|
|**max_concurrent_trials**|Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.|
|**enable_early_termination**|Whether to enable early termination if the score is not improving in the short term.|

### Specialized Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the .set_forecast_settings() method. 
The table below details the forecasting parameters we will be passing into our experiment.

|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**frequency**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.
|**cv_step_size**|Number of periods between two consecutive cross-validation folds. The default value is "auto", in which case AutoMl determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value.|
|**target_lags**|The target_lags specifies how far back we will construct the lags of the target variable.|
|**target_rolling_window_size**|The target_rolling_window_size specifies the size of a rolling window for aggregated lookback features.|

### Using lags and rolling window features
This training is also using the **target lags**, that is the previous values of the target variables, meaning the prediction uses a horizon. We therefore must still specify the `forecast_horizon` that the model will learn to forecast. The `target_lags` keyword specifies how far back we will construct the lags of the target variable, and the `target_rolling_window_size` specifies the size of the rolling window over which we will generate the `max`, `min` and `sum` features.

This notebook uses the .set_training(blocked_training_algorithms=...) parameter to exclude some models that take a longer time to train on this dataset.  You can choose to remove models from the blocked_training_algorithms list but you may need to increase the trial_timeout_minutes parameter value to get results.

Let's set up what we know about the dataset. 

**Target column** is what we want to forecast.

**Time column** is the time axis along which to predict.

In [10]:
# general job parameters
max_trials = 5
exp_name = "dpv2-bike-test"#forecasting-experiment"
target_column_name = "cnt"
time_column_name = "date"

### Setting forecaster maximum horizon 

The forecast horizon is the number of periods into the future that the model should predict. Here, we set the horizon to 14 periods (i.e. 14 days). Notice that this is much shorter than the number of days in the test set; we will need to use a rolling test to evaluate the performance on the whole test set. For more discussion of forecast horizons and guiding principles for setting them, please see the [energy demand notebook](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning/forecasting-energy-demand).  

In [11]:
forecast_horizon = 14

In [12]:
# Create the AutoML forecasting job with the related factory-function.
#######################mising param#############
#country_or_region_for_holidays="US",  # set country_or_region will trigger holiday featurizer
#cv_step_size="auto",
#n_cross_validations="auto", 
# Force the target column, to be integer type.

##########################################
forecasting_job = automl.forecasting(
    compute=cluster_name,
    # name="dpv2-forecasting-job-02",
    experiment_name=exp_name,
    training_data=my_training_data_input,
    # validation_data = my_validation_data_input,
    target_column_name=target_column_name,
    primary_metric="NormalizedRootMeanSquaredError",
    n_cross_validations=3,
    enable_model_explainability=True,
)

# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=60,
    trial_timeout_minutes=20, #20 change
    max_trials=max_trials,
    enable_early_termination=True,
    max_concurrent_trials=4,
)

# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    frequency="D",
    # target_lags="auto", #it is [12] in Nikolay 
    country_or_region_for_holidays="US",
    #enable_stack_ensemble=False,
    #target_rolling_window_size=4,
    #cv_step_size=1,# it should be auto
)

# Training properties are optional
forecasting_job.set_training(blocked_training_algorithms=["ExtremeRandomTrees"])

Class ForecastingJob: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### 2.2 Run the Command
Using the `MLClient` created earlier, we will now run this Command in the workspace.

In [13]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    forecasting_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Uploading training-mltable-folder (0.14 MBs): 100%|##########| 136269/136269 [00:01<00:00, 68295.22it/s]




Created job: ForecastingJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'target_column_name': 'cnt', 'weight_column_name': None, 'validation_data_size': None, 'cv_split_column_names': None, 'n_cross_validations': 3, 'test_data_size': None, 'task_type': <TaskType.FORECASTING: 'Forecasting'>, 'training_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/1c805cddcc0530df1fc249107d8c6d02/training-mltable-folder'}, 'validation_data': {'type': 'mltable'}, 'test_data': None, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'helpful_clock_sxgnphy4vg', 'description': None, 'tags': {}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/Azure/azureml-examples.git', 'mlflow.source.git.branch': 'sagoswami/update_bike_share_v2', 'mlflow.source.git.commit': '5de802c1e9f94feececb6faf5dd7f1bad0c63433', 'azureml.git.dirty': 'True'}, 'id': '/subscriptions/381

In [14]:
# Wait until AutoML training runs are finished
ml_client.jobs.stream(returned_job.name)

RunId: helpful_clock_sxgnphy4vg
Web View: https://ml.azure.com/runs/helpful_clock_sxgnphy4vg?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sagoswami_southcentralus_rg/workspaces/test

Execution Summary
RunId: helpful_clock_sxgnphy4vg
Web View: https://ml.azure.com/runs/helpful_clock_sxgnphy4vg?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sagoswami_southcentralus_rg/workspaces/test



## 5. Retrieve the Best Trial (Best Model's trial/run)
Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

### 5.1 Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    //pip install mlflow


#### Obtain the tracking URI for MLFlow

In [15]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

azureml://eastus2.api.azureml.ms/mlflow/v1.0/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sagoswami_southcentralus_rg/providers/Microsoft.MachineLearningServices/workspaces/test

Current tracking uri: azureml://eastus2.api.azureml.ms/mlflow/v1.0/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sagoswami_southcentralus_rg/providers/Microsoft.MachineLearningServices/workspaces/test


In [16]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### Get the AutoML parent Job

In [17]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "591640e8-0f88-49c5-adaa-39b9b9d75531"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'explained_variance': 0.1114027759125406,
 'mean_absolute_error': 661.0210884353742,
 'mean_absolute_percentage_error': 10.931066218715648,
 'median_absolute_error': 450.93333333333356,
 'normalized_mean_absolute_error': 0.08334649961358898,
 'normalized_median_absolute_error': 0.056857058798806405,
 'normalized_root_mean_squared_error': 0.11385744861162471,
 'normalized_root_mean_squared_log_error': 0.04995799377339727,
 'r2_score': 0.10792264429026525,
 'root_mean_squared_error': 903.0034249387954,
 'root_mean_squared_log_error': 0.1480328746365164,
 'spearman_correlation': 0.36915099580121574}, params={}, tags={'_azureml.ComputeTargetType': 'STANDARD_DS12_V2',
 'automl_best_child_run_id': 'helpful_clock_sxgnphy4vg_4',
 'fit_time': '<0.05379433333333333>;<0.041014666666666665>;<0.05559466666666666>;<0.04250933333333332>;<3>;',
 'iteration': '<1>;<0>;<2>;<3>;<4>;',
 'mlflow.rootRunId': 'helpful_clock_sxgnphy4vg',
 'mlflow.runName': 'helpful_c

In [18]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'model_explain_run': 'best_run', '_azureml.ComputeTargetType': 'STANDARD_DS12_V2', 'pipeline_id': '<__AutoML_SeasonalNaive__>;<__AutoML_Naive__>;<__AutoML_Average__>;<__AutoML_SeasonalAverage__>;<__AutoML_Ensemble__>;', 'score': '<0.11669651439395544>;<0.13636973162301635>;<0.36056973908167533>;<0.12192637204203947>;<0.11385744861162471>;', 'predicted_cost': '<0>;<0>;<0>;<0>;<0>;', 'fit_time': '<0.05379433333333333>;<0.041014666666666665>;<0.05559466666666666>;<0.04250933333333332>;<3>;', 'training_percent': '<100>;<100>;<100>;<100>;<100>;', 'iteration': '<1>;<0>;<2>;<3>;<4>;', 'run_preprocessor': '<>;<>;<>;<>;<>;', 'run_algorithm': '<SeasonalNaive>;<Naive>;<Average>;<SeasonalAverage>;<VotingEnsemble>;', 'automl_best_child_run_id': 'helpful_clock_sxgnphy4vg_4', 'model_explain_best_run_child_id': 'helpful_clock_sxgnphy4vg_4', 'mlflow.rootRunId': 'helpful_clock_sxgnphy4vg', 'mlflow.runName': 'helpful_clock_sxgnphy4vg', 'mlflow.user': 'Sampurna Goswami'}


### Get the AutoML best child run

In [19]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  helpful_clock_sxgnphy4vg_4
Best child run: 
<Run: data=<RunData: metrics={'explained_variance': 0.1114027759125406,
 'mean_absolute_error': 661.0210884353742,
 'mean_absolute_percentage_error': 10.931066218715648,
 'median_absolute_error': 450.93333333333356,
 'normalized_mean_absolute_error': 0.08334649961358898,
 'normalized_median_absolute_error': 0.056857058798806405,
 'normalized_root_mean_squared_error': 0.11385744861162471,
 'normalized_root_mean_squared_log_error': 0.04995799377339727,
 'r2_score': 0.10792264429026525,
 'root_mean_squared_error': 903.0034249387954,
 'root_mean_squared_log_error': 0.1480328746365164,
 'spearman_correlation': 0.36915099580121574}, params={}, tags={'mlflow.parentRunId': 'helpful_clock_sxgnphy4vg',
 'mlflow.rootRunId': 'helpful_clock_sxgnphy4vg',
 'mlflow.runName': 'goofy_cheese_7nchp071',
 'mlflow.source.name': 'automl_driver.py',
 'mlflow.source.type': 'JOB',
 'mlflow.user': 'Sampurna Goswami',
 'model_explain_run_id': '

### 5.2 Get best model run's metrics

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [20]:
pd.DataFrame(best_run.data.metrics, index=[0]).T

,0
root_mean_squared_log_error,0.148033
normalized_median_absolute_error,0.056857
mean_absolute_error,661.021088
spearman_correlation,0.369151
normalized_root_mean_squared_log_error,0.049958
explained_variance,0.111403
normalized_mean_absolute_error,0.083346
median_absolute_error,450.933333
normalized_root_mean_squared_error,0.113857
mean_absolute_percentage_error,10.931066


## 6. Model evaluation and deployemnt.
### 6.1 Download the best model locally

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [21]:
# Create local folder
import os

local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [22]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: c:\Users\sagoswami\projects\azureml-examples\sdk\python\jobs\automl-standalone-jobs\automl-forecasting-task-bike-share\artifact_downloads\outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [23]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

### Featurization
We can look at the engineered feature names generated in time-series featurization via. the JSON file named 'engineered_feature_names.json' under the run outputs.

In [24]:
import json

with open(os.path.join(local_path, "engineered_feature_names.json"), "r") as f:
    records = json.load(f)

records

['instant',
 'season',
 'yr',
 'mnth',
 'weekday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 '_automl_target_col_WASNULL',
 'instant_WASNULL',
 'season_WASNULL',
 'yr_WASNULL',
 'mnth_WASNULL',
 'weekday_WASNULL',
 'weathersit_WASNULL',
 'temp_WASNULL',
 'atemp_WASNULL',
 'hum_WASNULL',
 'windspeed_WASNULL',
 'casual_WASNULL',
 'registered_WASNULL',
 '_automl_year',
 '_automl_year_iso',
 '_automl_half',
 '_automl_quarter',
 '_automl_month',
 '_automl_day',
 '_automl_wday',
 '_automl_qday',
 '_automl_week',
 '_automl_IsPaidTimeOff',
 '_automl_Holiday_1 day after Christmas Day',
 '_automl_Holiday_1 day after Columbus Day',
 '_automl_Holiday_1 day after Independence Day',
 '_automl_Holiday_1 day after Labor Day',
 '_automl_Holiday_1 day after Martin Luther King, Jr. Day',
 '_automl_Holiday_1 day after Memorial Day',
 "_automl_Holiday_1 day after New Year's Day",
 '_automl_Holiday_1 day after Thanksgiving',
 '_automl_Holiday_1 day after Veterans Day

## Featurization

We can look at the engineered feature names generated in time-series featurization via. the JSON file named 'engineered_feature_names.json' under the run outputs. Note that a number of named holiday periods are represented. We recommend that you have at least one year of data when using this feature to ensure that all yearly holidays are captured in the training featurization.

#### View featurization summary
You can also see what featurization steps were performed on different raw features in the user data. For each raw feature in the user data, the following information is displayed:

+ Raw feature name
+ Number of engineered features formed out of this raw feature
+ Type detected
+ If feature was dropped
+ List of feature transformations for the raw feature

In [25]:
# Render the JSON as a pandas DataFrame
with open(os.path.join(local_path, "featurization_summary.json"), "r") as f:
    records = json.load(f)
fs = pd.DataFrame.from_records(records)

# View a summary of the featurization
fs[
    [
        "RawFeatureName",
        "TypeDetected",
        "Dropped",
        "EngineeredFeatureCount",
        "Transformations",
    ]
]

,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations
0,instant,Numeric,No,2,"[MedianImputer, ImputationMarker]"
1,season,Numeric,No,2,"[MedianImputer, ImputationMarker]"
2,yr,Numeric,No,2,"[MedianImputer, ImputationMarker]"
3,mnth,Numeric,No,2,"[MedianImputer, ImputationMarker]"
4,weekday,Numeric,No,2,"[MedianImputer, ImputationMarker]"
5,weathersit,Numeric,No,2,"[MedianImputer, ImputationMarker]"
6,temp,Numeric,No,2,"[MedianImputer, ImputationMarker]"
7,atemp,Numeric,No,2,"[MedianImputer, ImputationMarker]"
8,hum,Numeric,No,2,"[MedianImputer, ImputationMarker]"
9,windspeed,Numeric,No,2,"[MedianImputer, ImputationMarker]"


In [ ]:
# from azure.ai.ml.entities import (
#     Environment,
#     BatchEndpoint,
#     BatchDeployment,
#     BatchRetrySettings,
#     Model,
# )
# from azure.ai.ml.constants import BatchDeploymentOutputAction
# from sklearn.externals import joblib

# fitted_model = joblib.load("./artifact_downloads/outputs/model.pkl")

## 6.2 Forecasting using batch endpoint<a id="forecast"></a>

Now that we have retrieved the best pipeline/model, it can be used to make predictions on test data. We will do batch scoring on the test dataset which should have the same schema as training dataset.

The inference will run on a remote compute. In this example, it will re-use the training compute.
Fisrt we will load model and environment from the local file.

### Create a model endpoint
First we need to register the model, environment and batch endpoint.

In [26]:
from azure.ai.ml.entities import (
    Environment,
    BatchEndpoint,
    BatchDeployment,
    BatchRetrySettings,
    Model,
)
from azure.ai.ml.constants import BatchDeploymentOutputAction

model_name = "bike-share"
batch_endpoint_name = "bike-share"

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description="Bike share model.",
)
registered_model = ml_client.models.create_or_update(model)

env = Environment(
    name="automl-tabular-env",
    description="environment for automl inference",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    conda_file="artifact_downloads/outputs/conda_env_v_1_0_0.yml",
)

endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="this is a sample batch endpoint",
)
ml_client.begin_create_or_update(endpoint)

BatchEndpoint({'scoring_uri': 'https://bike-share.eastus2.inference.ml.azure.com/jobs', 'swagger_uri': None, 'provisioning_state': 'Succeeded', 'name': 'bike-share', 'description': 'this is a sample batch endpoint', 'tags': {}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'azureml.onlineendpointid': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sagoswami_southcentralus_rg/providers/microsoft.machinelearningservices/workspaces/test/onlineendpoints/bike-share', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/be:104cbf67-baa5-4a37-bc93-451a3fa51799:3b054981-4fc6-45bd-ba18-fb3a26a5e6e9?api-version=2022-05-01'}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sagoswami_southcentralus_rg/providers/Microsoft.MachineLearningServices/workspaces/test/batchEndpoints/bike-share', 'Resource__so

Create a batch deployment we will use the forecasting_script.py which will load the model and will call forecast each time we will envoke the endpoint.

In [27]:
output_file = "forecast.json"
batch_deployment = BatchDeployment(
    name="non-mlflow-deployment-1",
    description="this is a sample non-mlflow deployment",
    endpoint_name=batch_endpoint_name,
    model=registered_model,
    code_path="./forecast",
    scoring_script="forecasting_script.py",
    environment=env,
    environment_variables={
        "TARGET_COLUMN_NAME": target_column_name,
    },
    compute=cluster_name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name=output_file,
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
)

Finally, start a model deployment.

In [28]:
ml_client.begin_create_or_update(batch_deployment)

BatchDeployment({'endpoint_name': 'bike-share', 'type': None, 'name': 'non-mlflow-deployment-1', 'description': 'this is a sample non-mlflow deployment', 'tags': {}, 'properties': {}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sagoswami_southcentralus_rg/providers/Microsoft.MachineLearningServices/workspaces/test/batchEndpoints/bike-share/deployments/non-mlflow-deployment-1', 'Resource__source_path': None, 'base_path': 'c:\\Users\\sagoswami\\projects\\azureml-examples\\sdk\\python\\jobs\\automl-standalone-jobs\\automl-forecasting-task-bike-share', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x0000025510DDCC70>, 'model': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sagoswami_southcentralus_rg/providers/Microsoft.MachineLearningServices/workspaces/test/models/bike-share/versions/13', 'code_configuration': <azure.ai.ml.entities._deployment.code_configuration.CodeConfiguration object at 0x0000025510

We need to create the Input, representing URI folder, because the batch endpoint is intended to process multiple files at a time. In this example we will use only one test file, we have uploaded to the blob storage before. This file must be available through the url link.

Create an inference job.

In [29]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=batch_endpoint_name,
    input=my_test_data_input,
    deployment_name="non-mlflow-deployment-1",  # name is required as default deployment is not set
)

Uploading test_dataset (0.06 MBs): 100%|##########| 63760/63760 [00:01<00:00, 40672.32it/s]




We will stream the job output to monitor the execution.

In [30]:
job_name = job.name
batch_job = ml_client.jobs.get(name=job_name)
print(batch_job.status)
# stream the job logs
ml_client.jobs.stream(name=job_name)

Running
RunId: 86c59301-15fe-4842-98a8-c4b62c09e921
Web View: https://ml.azure.com/runs/86c59301-15fe-4842-98a8-c4b62c09e921?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sagoswami_southcentralus_rg/workspaces/test

Streaming logs/azureml/executionlogs.txt

[2022-10-10 15:29:34Z] Submitting 1 runs, first five are: 7ae63bb9:dfcd7410-8e26-41f3-a243-3f582a02e395
[2022-10-10 15:38:25Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: 86c59301-15fe-4842-98a8-c4b62c09e921
Web View: https://ml.azure.com/runs/86c59301-15fe-4842-98a8-c4b62c09e921?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sagoswami_southcentralus_rg/workspaces/test


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "message_format": "Pipeline has some failed steps. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus2",
    "location": "eastus2",
    "time": "2022-10-10T15:38:25.813264Z",
    "component_name": ""
} 

### Download the prediction result for metrics calculation
The output of prediction is saved in JSON format. You can download it and calculation some error metrics for the forecasts and vizualize the predictions vs. the actuals.

In [ ]:
ml_client.jobs.download(job_name, download_path=".")

In [ ]:
fcst_df = pd.read_json(output_file, orient="table")
fcst_df.head()

Calculate the metrics.

In [ ]:
from metrics_helper import calculate_metrics
calculate_metrics(fcst_df[target_column_name], fcst_df["predicted"])

#### Forecast versus actuals plot.
We will join the last three forecast horizons of history data with the predictions to evaluate the forecast quality on forecast vs versus actuals plot.

In [ ]:
history_data = pd.read_csv(
    "./data/training-mltable-folder/nyc_energy_training_clean.csv",
    parse_dates=[time_column_name],
)
history_data.sort_values(by=time_column_name, inplace=True)
history_data = history_data.iloc[-3 * forecast_horizon :]
# Merge predictions to historic data.
df = pd.concat([history_data, fcst_df], sort=False, ignore_index=True)
df.set_index(time_column_name, inplace=True)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(df[[target_column_name, "predicted"]])
plt.xticks(rotation=45)
plt.title(f"Predicted vs. Actuals")
plt.legend(["actual", "forecast"])
plt.show()

############################################### old #########################

## Evaluate

We now use the best fitted model from the AutoML Run to make forecasts for the test set. We will do batch scoring on the test dataset which should have the same schema as training dataset.

The scoring will run on a remote compute. In this example, it will reuse the training compute.

In [ ]:
test_experiment = Experiment(ws, experiment_name + "_test")

### Retrieving forecasts from the model
To run the forecast on the remote compute we will use a helper script: forecasting_script. This script contains the utility methods which will be used by the remote estimator. We copy the script to the project folder to upload it to remote compute.

In [ ]:
import os
import shutil

script_folder = os.path.join(os.getcwd(), "forecast")
os.makedirs(script_folder, exist_ok=True)
shutil.copy("forecasting_script.py", script_folder)

For brevity, we have created a function called run_forecast that submits the test data to the best model determined during the training run and retrieves forecasts. The test set is longer than the forecast horizon specified at train time, so the forecasting script uses a so-called rolling evaluation to generate predictions over the whole test set. A rolling evaluation iterates the forecaster over the test set, using the actuals in the test set to make lag features as needed. 

In [ ]:
from run_forecast import run_rolling_forecast

remote_run = run_rolling_forecast(
    test_experiment, compute_target, best_run, test, target_column_name
)
remote_run

In [ ]:
remote_run.wait_for_completion(show_output=False)

### Download the prediction result for metrics calculation
The test data with predictions are saved in artifact outputs/predictions.csv. You can download it and calculation some error metrics for the forecasts and vizualize the predictions vs. the actuals.

In [ ]:
remote_run.download_file("outputs/predictions.csv", "predictions.csv")
df_all = pd.read_csv("predictions.csv")

In [ ]:
from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from sklearn.metrics import mean_absolute_error, mean_squared_error
from matplotlib import pyplot as plt

# use automl metrics module
scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all["predicted"],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET),
)

print("[Test data scores]\n")
for key, value in scores.items():
    print("{}:   {:.3f}".format(key, value))

# Plot outputs
%matplotlib inline
test_pred = plt.scatter(df_all[target_column_name], df_all["predicted"], color="b")
test_test = plt.scatter(
    df_all[target_column_name], df_all[target_column_name], color="g"
)
plt.legend(
    (test_pred, test_test), ("prediction", "truth"), loc="upper left", fontsize=8
)
plt.show()

For more details on what metrics are included and how they are calculated, please refer to [supported metrics](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-understand-automated-ml#regressionforecasting-metrics). You could also calculate residuals, like described [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-understand-automated-ml#residuals).


Since we did a rolling evaluation on the test set, we can analyze the predictions by their forecast horizon relative to the rolling origin. The model was initially trained at a forecast horizon of 14, so each prediction from the model is associated with a horizon value from 1 to 14. The horizon values are in a column named, "horizon_origin," in the prediction set. For example, we can calculate some of the error metrics grouped by the horizon:

In [ ]:
from metrics_helper import MAPE, APE

df_all.groupby("horizon_origin").apply(
    lambda df: pd.Series(
        {
            "MAPE": MAPE(df[target_column_name], df["predicted"]),
            "RMSE": np.sqrt(
                mean_squared_error(df[target_column_name], df["predicted"])
            ),
            "MAE": mean_absolute_error(df[target_column_name], df["predicted"]),
        }
    )
)

To drill down more, we can look at the distributions of APE (absolute percentage error) by horizon. From the chart, it is clear that the overall MAPE is being skewed by one particular point where the actual value is of small absolute value.

In [ ]:
df_all_APE = df_all.assign(APE=APE(df_all[target_column_name], df_all["predicted"]))
APEs = [
    df_all_APE[df_all["horizon_origin"] == h].APE.values
    for h in range(1, forecast_horizon + 1)
]

%matplotlib inline
plt.boxplot(APEs)
plt.yscale("log")
plt.xlabel("horizon")
plt.ylabel("APE (%)")
plt.title("Absolute Percentage Errors by Forecast Horizon")

plt.show()

In [ ]:

# full_df.sort_values(
#     by = ['new_date'],
#     axis = 0, 
#     ascending = True,
#     inplace=  True)
# # # Get the date from pad.Date
# # full_df['month'] = pd.DatetimeIndex(full_df['new_date']).month 
# # full_df['day'] = pd.DatetimeIndex(full_df['new_date']).day 
# # full_df['year'] = pd.DatetimeIndex(full_df['new_date']).year

# # For timestamp
# # for index, row in full_df.iterrows():
# #     year = row['year']
# #     month = row['month']
# #     day = row['day']
# #     print(year," ",month," ",day)
# #     full_df['date_stamp'][index]= pd.Timestamp(year=year, month=month, day=day, hour=12)

# # train_df['date'] = train_df['date_stamp'] 
# # test_df['date'] = test_df['date_stamp'] 